<a href="https://colab.research.google.com/github/egemengulpinar/Image-Super-Resolution-SRCNN-notes/blob/main/basic-practice-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchsummary import summary

In [ ]:
#DEFINE YOUR DEVICE
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device) #if cpu, go Runtime-> Change runtime type-> Hardware accelerator GPU -> Save -> Redo previous steps

In [ ]:
#DOWNLOAD CIFAR-10 DATASET
train_data = datasets.CIFAR10('./data', train = True, download = True, transform = transforms.ToTensor())

test_data = datasets.CIFAR10('./data', train = False, transform = transforms.ToTensor())

In [ ]:
#DEFINE DATA GENERATOR
batch_size = 100
train_generator = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)

test_generator = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [ ]:
model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
            nn.BatchNorm2d(256),

            nn.Flatten(),
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))

In [ ]:
#CREATE MODEL
# model = CNN()
model.to(device)
summary(model,(3,32,32))

In [ ]:
#DEFINE LOSS FUNCTION AND OPTIMIZER
learning_rate = 0.001
# momentum = 0.95
loss_fun = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#optimizer=torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [ ]:
#TRAIN THE MODEL
def train(epoch):
  print('\nEpoch : %d'%epoch)
  model.train()
  epoch = 1
  train_losses = []
  num_of_batch=np.int(len(train_generator.dataset)/batch_size)
  running_loss=0
  loss_values = np.zeros(epoch*num_of_batch)
  for i in range(epoch):
    for batch_idx, (x_train, y_train) in enumerate(train_generator):
      x_train, y_train = x_train.to(device), y_train.to(device)
      optimizer.zero_grad()
      y_pred = model(x_train)
      loss = loss_fun(y_pred, y_train)
      loss_values[num_of_batch*i+batch_idx] = loss.item()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      if (batch_idx+1) % batch_size == 0:
          print('Epoch: {}/{} [Batch: {}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              i+1, epoch, (batch_idx+1) * len(x_train), len(train_generator.dataset),
              100. * (batch_idx+1) / len(train_generator), loss.item()))
      train_loss=running_loss/len(train_generator)
      train_losses.append(train_loss)

  return loss_values

In [ ]:
#TEST THE MODEL
def test(epoch):
  model.eval()
  correct=0
  total=0

  for x_val, y_val in test_generator:
    x_val = x_val.to(device)
    y_val = y_val.to(device)

    output = model(x_val)
    y_pred = output.argmax(dim=1)
    # print("Output --> ", output)
    # print("y_pred --> ", y_pred)
    # print("x_val --> ", x_val)
    # print("y_val -->" , y_val)

    for i in range(y_pred.shape[0]):

      if y_val[i]==y_pred[i]:
        correct += 1
      total +=1

  print('Validation accuracy: %.2f%%' %((100*correct)//(total)))

In [ ]:
epochs=10
for epoch_iterate in range(1,epochs+1):
  return_loss_values = train(epoch_iterate)
  test(epoch_iterate)

In [ ]:
#PLOT THE LEARNING CURVE
num_of_batch=np.int(len(train_generator.dataset)/batch_size)
iterations = np.linspace(0,5,num_of_batch*5)
plt.plot(return_loss_values, '-o')
# plt.plot(iterations,test_loss)
plt.title('Validation & Train Losses')
plt.xlabel('Epoch')
plt.legend(['Valid','Train'])
plt.ylabel('Loss')
plt.grid('on')

plt.show()